In [22]:
import pandas as pd
import os
import json
import numpy as np
from functions import Convertor_Single
from functions import Convertor_Multiple
from functions import df_info
from functions import df_process_num
from functions import df_reshape
from functions import random_forest_classifier
from functions import df_pivot_multiple
from functions import pivot_diff_lag
from functions import merge_json_data
from functions import retrieve_and_merge_data
from functions import num_to_cik

## STEP01: Converting to CSV

In [2]:
df = pd.read_csv('Submissions.csv')

In [3]:
sic_2834 = df[df['SIC']==2834]

In [4]:
num_list = list(sic_2834['CIK'])

In [5]:
new_list = num_to_cik(num_list)

In [6]:
len(new_list)

1087

In [ ]:
merged_2834 = Convertor_Multiple(new_list)

In [13]:
len(merged_2834['CIK'].unique())

1024

In [ ]:
merged_2834.to_csv('merged_2834.csv',index=False)


In [3]:
merged_2834 = pd.read_csv('merged_2834.csv')

In [5]:
df_num = df_process_num(merged_2834)

In [4]:
df_info(merged_2834)

(0.99, (39825, 5746))

In [7]:
df_info(df_num)

(0.0, (26883, 5741))

In [8]:
asd = df_num[:200]

In [15]:
import pandas as pd

def drop_columns_with_missing_values(df, threshold):
    """
    Drops columns from the DataFrame where the proportion of missing values is greater than the specified threshold.

    Parameters:
    df (pd.DataFrame): The input DataFrame.
    threshold (float): The threshold proportion of missing values to drop the column. Default is 0.7.

    Returns:
    pd.DataFrame: A DataFrame with the specified columns dropped.
    """
    # Calculate the proportion of missing values for each column
    missing_proportion = df.isnull().mean()
    
    # Identify columns that have missing value proportion greater than the threshold
    columns_to_drop = missing_proportion[missing_proportion > threshold].index
    
    # Drop the identified columns
    df_dropped = df.drop(columns=columns_to_drop)
    
    return df_dropped

# Example usage:
# Assuming 'data' is your existing DataFrame
# cleaned_data = drop_columns_with_missing_values(data)


In [16]:
cleaned_data = drop_columns_with_missing_values(merged_2834,0.9)

In [18]:
df_info(cleaned_data)

(0.7, (39825, 144))

In [19]:
df_num = df_process_num(cleaned_data)

In [ ]:
lagged = pivot_diff_lag(df_num,'CIK','Frame',3)

In [33]:
lagged=lagged.reset_index()

In [41]:
import pandas as pd
import numpy as np

def drop_inf_rows(df):
    """
    Drops rows from the DataFrame that contain -inf or +inf values.

    Parameters:
    df (pd.DataFrame): The input DataFrame.

    Returns:
    pd.DataFrame: A DataFrame with the rows containing infinite values dropped.
    """
    # Replace -inf and +inf with NaN, then drop rows with NaN values
    df_replaced = df.replace([np.inf, -np.inf], np.nan)
    
    return df_replaced

# Example usage:
# Assuming 'data' is your existing DataFrame
# cleaned_data = drop_inf_rows(data)


In [42]:
lag_clean = drop_inf_rows(lagged)

In [43]:
lag_clean

,CIK,Frame,AccountsPayableAndAccruedLiabilitiesCurrent,AccountsPayableAndAccruedLiabilitiesCurrent_1,AccountsPayableAndAccruedLiabilitiesCurrent_2,AccountsPayableAndAccruedLiabilitiesCurrent_3,AccountsPayableCurrent,AccountsPayableCurrent_1,AccountsPayableCurrent_2,AccountsPayableCurrent_3,...,WeightedAverageNumberOfDilutedSharesOutstanding_2,WeightedAverageNumberOfDilutedSharesOutstanding_3,WeightedAverageNumberOfShareOutstandingBasicAndDiluted,WeightedAverageNumberOfShareOutstandingBasicAndDiluted_1,WeightedAverageNumberOfShareOutstandingBasicAndDiluted_2,WeightedAverageNumberOfShareOutstandingBasicAndDiluted_3,WeightedAverageNumberOfSharesOutstandingBasic,WeightedAverageNumberOfSharesOutstandingBasic_1,WeightedAverageNumberOfSharesOutstandingBasic_2,WeightedAverageNumberOfSharesOutstandingBasic_3
0,1800,CY2012Q1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.00,-1.00,NaN,NaN,NaN,NaN,NaN,-0.00,0.00,-1.00
1,1800,CY2012Q2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-1.00,NaN,NaN,NaN,NaN,NaN,-0.0,0.00,-1.00,NaN
2,1800,CY2012Q3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,-0.01,NaN,NaN,NaN,NaN,0.0,-1.00,NaN,-0.01
3,1800,CY2012Q4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.01,-0.01,NaN,NaN,NaN,NaN,-1.0,NaN,-0.01,-0.01
4,1800,CY2013Q1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.01,-1.00,NaN,NaN,NaN,NaN,NaN,-0.01,-0.01,-1.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26878,1971543,CY2022Q3,NaN,NaN,NaN,NaN,NaN,NaN,-1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.00,NaN,NaN
26879,1971543,CY2022Q4,NaN,NaN,NaN,NaN,NaN,-1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,-1.0,NaN,NaN,NaN
26880,1971543,CY2023Q1,NaN,NaN,NaN,NaN,-1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26881,1971543,CY2023Q2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [44]:
df_dropped = lag_clean.dropna(how='all')

In [46]:
df_dropped.to_csv('2934_processed.csv',index=False)